# Stock Profile Service - Detailed Listing Information

Learn how to fetch comprehensive listing details for individual stocks including IPO date, listing status, foreign ownership limits, and ISIN codes.

## Overview

The **Stock Profile Service** provides detailed listing information for stocks. Essential for:

- **Compliance & Regulation**: Track foreign ownership limits and restrictions
- **Trading Eligibility**: Verify listing status and first trade date
- **Corporate Actions**: Monitor par value and listed shares
- **ISIN Lookups**: Get international securities identification numbers
- **IPO Research**: Access IPO price and listing date information
- **Warrant Analysis**: Track exercise price and ratio for warrants

### When to Use This Service

- Foreign investor compliance (checking foreign limits)
- Trading system setup (ISIN codes, market classification)
- IPO analysis and new listing monitoring
- Free float calculations
- Warrant and derivative trading

## Installation

In [1]:
!pip install settfex

zsh:1: command not found: pip


## Import Libraries

In [2]:
import asyncio
from settfex.services.set import get_profile
import pandas as pd

print("Libraries imported successfully!")

Libraries imported successfully!


## Basic Usage

In [3]:
# Fetch stock profile
profile = await get_profile("PTT")

print(f"Symbol: {profile.symbol}")
print(f"Company: {profile.name_en}")
print(f"Thai Name: {profile.name_th}")
print(f"\nMarket Classification:")
print(f"  Market: {profile.market}")
print(f"  Sector: {profile.sector_name} ({profile.sector_code})")
print(f"  Industry: {profile.industry_name} ({profile.industry_code})")
print(f"\nListing Details:")
print(f"  Listed Date: {profile.listed_date}")
print(f"  First Trade: {profile.first_trade_date}")
print(f"  IPO Price: {profile.ipo} {profile.currency}")
print(f"  Par Value: {profile.par_value} {profile.currency}")
print(f"\nForeign Ownership:")
print(f"  Foreign Limit: {profile.foreign_limit}%")
print(f"  Foreign Room: {profile.foreign_room:,} shares")
print(f"  Available: {profile.available_foreign:,} shares")

2025-10-05 17:37:40 | INFO     | settfex.services.set.stock.profile_stock:__init__:126 | StockProfileService initialized with base_url=https://www.set.or.th
2025-10-05 17:37:40 | INFO     | settfex.services.set.stock.profile_stock:fetch_profile:166 | Fetching profile data for symbol 'PTT' (lang=en) from https://www.set.or.th/api/set/stock/PTT/profile?lang=en
2025-10-05 17:37:40 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:37:40 | INFO     | settfex.utils.session_manager:__init__:98 | SessionManager created with browser=chrome120, warmup_site=set, cache=enabled
2025-10-05 17:37:40 | INFO     | settfex.utils.session_cache:__init__:76 | SessionCache initialized at /Users/sarat/.settfex/cache (ttl=3600s, size_limit=100MB)
2025-10-05 17:37:40 | SUCCESS  | settfex.utils.session_manager:_try_load_from_cache:202 | ✓ Loaded session from cache: 4 cookies (age=398s, browser=chrome120)
2025-10

Symbol: PTT


AttributeError: 'StockProfile' object has no attribute 'name_en'

## Advanced Usage - Batch Processing

In [4]:
# Fetch multiple stock profiles
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL"]
tasks = [get_profile(symbol) for symbol in symbols]
profiles = await asyncio.gather(*tasks)

# Create comparison DataFrame
data = [
    {
        "symbol": p.symbol,
        "name": p.name_en,
        "market": p.market,
        "sector": p.sector_name,
        "listed_date": p.listed_date,
        "ipo_price": p.ipo,
        "foreign_limit": p.foreign_limit,
        "free_float": p.free_float_percent
    }
    for p in profiles
]

df = pd.DataFrame(data)
print("\nStock Profile Comparison:")
print(df.to_string(index=False))

2025-10-05 17:38:06 | INFO     | settfex.services.set.stock.profile_stock:__init__:126 | StockProfileService initialized with base_url=https://www.set.or.th
2025-10-05 17:38:06 | INFO     | settfex.services.set.stock.profile_stock:fetch_profile:166 | Fetching profile data for symbol 'PTT' (lang=en) from https://www.set.or.th/api/set/stock/PTT/profile?lang=en
2025-10-05 17:38:06 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:38:06 | INFO     | settfex.services.set.stock.profile_stock:__init__:126 | StockProfileService initialized with base_url=https://www.set.or.th
2025-10-05 17:38:06 | INFO     | settfex.services.set.stock.profile_stock:fetch_profile:166 | Fetching profile data for symbol 'KBANK' (lang=en) from https://www.set.or.th/api/set/stock/KBANK/profile?lang=en
2025-10-05 17:38:06 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=

AttributeError: 'StockProfile' object has no attribute 'name_en'

## Financial Firm Use Case: Foreign Ownership Monitoring

In [ ]:
async def check_foreign_room(symbols: list[str], min_room_shares: int = 1_000_000):
    """
    Check which stocks have available foreign room for foreign investors.
    Critical for foreign fund managers and international investors.
    """
    tasks = [get_profile(symbol) for symbol in symbols]
    profiles = await asyncio.gather(*tasks, return_exceptions=True)
    
    available = []
    restricted = []
    
    for profile in profiles:
        if isinstance(profile, Exception):
            continue
        
        if profile.available_foreign >= min_room_shares:
            available.append({
                "symbol": profile.symbol,
                "name": profile.name_en,
                "foreign_limit": profile.foreign_limit,
                "available_shares": profile.available_foreign,
                "foreign_room": profile.foreign_room
            })
        else:
            restricted.append({
                "symbol": profile.symbol,
                "name": profile.name_en,
                "available_shares": profile.available_foreign
            })
    
    print(f"✓ Available for foreign purchase ({len(available)} stocks):")
    if available:
        av_df = pd.DataFrame(available)
        print(av_df.to_string(index=False))
    
    print(f"\n✗ Limited/No foreign room ({len(restricted)} stocks):")
    if restricted:
        res_df = pd.DataFrame(restricted)
        print(res_df.to_string(index=False))

# Check foreign room for blue chips
await check_foreign_room(["PTT", "KBANK", "CPALL", "AOT", "BBL", "SCB"], min_room_shares=1_000_000)

## Use Case: IPO Analysis

In [ ]:
# Analyze IPO pricing vs current market
from settfex.services.set import get_highlight_data

async def ipo_performance_analysis(symbol: str):
    """
    Compare IPO price to current market metrics.
    """
    # Fetch both profile and current data
    profile, highlight = await asyncio.gather(
        get_profile(symbol),
        get_highlight_data(symbol)
    )
    
    # Calculate returns since IPO (simplified - would need current price)
    print(f"IPO Analysis for {profile.name_en}:")
    print(f"  Symbol: {profile.symbol}")
    print(f"  Listed Date: {profile.listed_date}")
    print(f"  IPO Price: {profile.ipo} {profile.currency}")
    print(f"  Par Value: {profile.par_value} {profile.currency}")
    print(f"\nCurrent Valuation:")
    print(f"  Market Cap: {highlight.market_cap:,.0f} THB")
    print(f"  P/E Ratio: {highlight.pe_ratio}")
    print(f"  P/B Ratio: {highlight.pb_ratio}")

await ipo_performance_analysis("CPALL")

## Error Handling

In [ ]:
async def safe_get_profile(symbol: str):
    try:
        profile = await get_profile(symbol)
        print(f"✓ {symbol}: {profile.name_en}")
        return profile
    except Exception as e:
        print(f"✗ {symbol}: {e}")
        return None

await safe_get_profile("PTT")
await safe_get_profile("INVALID")

## Next Steps

- **[Stock List](01_stock_list.ipynb)** - Get all stock symbols
- **[Highlight Data](02_highlight_data.ipynb)** - Market metrics
- **[Company Profile](04_company_profile.ipynb)** - Company details and governance
- **[Shareholder](06_shareholder.ipynb)** - Ownership structure

**Documentation**: `/docs/settfex/services/set/profile_stock.md`